In [0]:
from pyspark.sql.types import StructField,StructType,StringType,IntegerType

students_data = [(1, "Hitachi"),(2, "Hyundai"),(3, "Bajaj"),
                (4, "Hero"),(5, "Tata")]

students_schema =StructType([StructField("id", IntegerType(), True),
                            StructField("student", StringType(), True)])


student_df = spark.createDataFrame(data=students_data, schema=students_schema)
student_df.printSchema()
student_df.show()


root
 |-- id: integer (nullable = true)
 |-- student: string (nullable = true)

+---+-------+
| id|student|
+---+-------+
|  1|Hitachi|
|  2|Hyundai|
|  3|  Bajaj|
|  4|   Hero|
|  5|   Tata|
+---+-------+



In [0]:
from pyspark.sql.functions import col,lag,lead,when
from pyspark.sql.window import Window as wd

windowSpec = wd.orderBy("id")

student_df = student_df.withColumn("prev_student",lag("student").over(windowSpec))\
                        .withColumn("next_student",lead("student").over(windowSpec))\
                        .withColumn("new_student",when((col("id")%2) == 0,col("prev_student"))
                                            .when((col("id")%2) != 0,col("next_student"))
                                            .when(col("id").isNull(),col("student")))\
                        .select("id", "student","new_student")
                        
                        
                                             
                           
student_df.show()

+---+-------+-----------+
| id|student|new_student|
+---+-------+-----------+
|  1|Hitachi|    Hyundai|
|  2|Hyundai|    Hitachi|
|  3|  Bajaj|       Hero|
|  4|   Hero|      Bajaj|
|  5|   Tata|       null|
+---+-------+-----------+

